In [1]:
import importlib
import src.chat_bot
importlib.reload(src.chat_bot)
from src.chat_bot import ExconManual

In [2]:
excon = ExconManual("")

In [ ]:
model_to_use="gpt-3.5-turbo"
#model_to_use="gpt-4"
#model_to_use="gpt-3.5-turbo-16k"

In [16]:
#user_context = "What is a CFC Account?"
#user_context = "I am submitting documentation to the financial surveillance department. What information do I need to include in my submission?" # (A.3 (B))
#user_context = "Can a bank provide foreign currency denominated facilities to a South African Corporate?" #  (A.3 (B))
#user_context = "For how long does the approval for a facility remain valid?" #  (A.3 (B))
#user_context = "Other than authorised transaction requests, are there requirements for authorised dealers to submit information to the financial surveillance department?" #  (A.3 (B))
#user_context = "I have been asked to act on an approval issued to another bank. Can I do that?" #  (A.3 (B))
#####user_context = "--- Same as above but use "Authorised Dealer" instead of "Bank"
#user_context = "If I am concerned that as client may in breach of regulations, what should I do?" #  (A.3 (B))
#user_context = "What constitutes 'documentary evidence' for a foreign currency transaction?" #  (A.3 (C))
#user_context = "Is Swaziland part of the CMA?" #  (A.3 (E))
#user_context = "A client wants to buy Namibian Dollars. What approvals do they need before they can do this?" #  (A.3 (E))
#user_context = "A person from Lesotho wants to buy dollars from me. Can I sell them the dollars?" #  (A.3 (E))
#user_context = "A person from Namibia wants to buy rand from me. Can I sell them the rand?" #  (A.3 (E))
#user_context = "A treasury management company wants to trade dollars with me. What do I need to check before I can do this?" #  (A.4 (B))
#user_context = "A client wants to import goods and pay in foreign currency. What documentation do I need before I can assist with the transaction" #  (B.1 (B))
#user_context = "Over and above the price of the goods, what additional costs can be covered in foreign currency?" #  (B.1 (C))
#user_context = "Can advance payment be made in a foreign currency to cover imported goods?" #  (B.1 (D)(i))

#user_context = "Can advance payment be made in a foreign currency to cover imported capital goods?" #  (B.1 (D)(ii))
#user_context = "When paying for imports in cash, what documentation is required before providing the foreign currency?" #  (B.1 (D)(iii))

#user_context = "Can imported goods be financed over more than 12 months?" #  (B.1 (D)(iv))
#user_context = "How much money can an individual invest internationally?" #  ?
#user_context = "How much money can an individual take offshore?" #  ?

#user_context = "A media company want to list offshore. Are there specific conditions for this?"
# user_context = "How many rands can a resident take offshore in cash?"
#user_context = "Can I grant a foreign company a Rand facility to pay for goods they wish to purchase from a South African exporter?"
#user_context = "Can I support a non-residential entity with short-term rand trade finance for their exports from South Africa?"
#user_context = "Hi"
#user_context = "Who can trade gold?"
#user_context = "are there specific exchange control rules for religious organizations?"
#user_context = "How much foreign currency can a charity take offshore each year?"

user_context = "pensions"

excon.user_provides_input(user_context, threshold=0.15, model_to_use=model_to_use, temperature=0, max_tokens=300)

print(f'{excon.messages[-1]["role"]}: {excon.messages[-1]["content"]}')



INFO:src.chat_bot:Entering RAG with query: pensions
INFO:src.chat_bot:#################   Similarity Search       #################
INFO:src.chat_bot:--   No relevant definitions found
INFO:src.chat_bot:--   Relevant Sections
INFO:src.chat_bot:0.1053:              B.10(J):         heading: Insurance and pensions. Pension and provident schemes
INFO:src.chat_bot:0.1138:              B.10(A):         heading: Insurance and pensions. Introduction
INFO:src.chat_bot:0.1231:              B.10(B):         heading: Insurance and pensions. Glossary of terms
INFO:src.chat_bot:0.1265:              B.10(H):         heading: Insurance and pensions. Long-term insurance by residents
INFO:src.chat_bot:0.1346:              B.10(E):         heading: Insurance and pensions. Short-term insurance by residents
INFO:src.chat_bot:0.1390:          B.10(D)(ii):         heading: Insurance and pensions. General. Branch settlements
INFO:src.chat_bot:0.1393:              B.10(I):         heading: Insurance and pensi

NameError: name 'modified_section_to_add' is not defined

In [25]:
excon.system_state = excon.system_states[0]
print(f'System State = {excon.system_state}')


System State = rag


In [30]:
flag, response = excon.resource_augmented_query(user_context, model_to_use = model_to_use, temperature = 0.0, max_tokens = 512, df_definitions = df_definitions, df_search_sections = df_search_sections)
print(f"Flag: {flag}")
print(f'Response: {response}')


{'role': 'system', 'content': "You are attempting to answer questions from an Authorised Dealer based only on the relevant documentation provided. You have only three options:\n1) Answer the question. If you do this, your must preface to response with the word 'ANSWER:'. If possible also provide a reference to the relevant documentation for the user to cross-check. Use this if you are sure about your answer.\n2) Request additional documentation. If, in the body of the relevant documentation, is a reference to another section of the document that is directly relevant, respond with the word 'SECTION:' followed by the section reference which looks like A.1(A)(i)(aa). \n3) State 'NONE:' (and nothing else) in all cases where you are not confident about either of the first two options\nPotentially relevant document section(s):\nIntroduction \n    In terms of Regulation 2(2), an Authorised Dealer shall not buy, borrow, receive, sell, lend or deliver any foreign currency or gold except for suc

In [20]:
print(excon.get_regulation_detail('C.(C)'))
#excon.get_regulation_detail('B.18(B)(i)(b)')

C. Gold
&nbsp;&nbsp;&nbsp;&nbsp;(C) Acquisition of gold for trade purposes
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(i) The acquisition of gold for legitimate trade purposes by e.g. manufacturing jewellers, dentists, is subject to the approval of the South African Diamond and Precious Metals Regulator.
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(ii) After receiving such approval, a permit must be obtained from SARS which will entitle the permit holder to approach Rand Refinery Limited for an allocation of gold.
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(iii) The holders of gold, having received the approvals outlined above, are exempt from the provisions of Regulation 5(1).


In [3]:
excon_test = ExconManual(log_file='',input_folder = "./inputs_test/")
#excon_test = ExconManual(log_file='',input_folder = "./inputs/")


In [4]:
#user_context = "How much money can an individual take offshore in any year?"
user_context = "Who can trade gold"
relevant_definitions, relevant_sections = excon_test.similarity_search(user_context)


# response = excon_test.user_provides_input(user_context=user_context,
#                                           threshold=0.15,
#                                           model_to_use="gpt-3.5-turbo", 
#                                           temperature = 0, 
#                                           max_tokens = 200, 
#                                           df_definitions = relevant_definitions, 
#                                           df_search_sections = relevant_sections)
# print(response)


INFO:src.chat_bot:#################   Similarity Search       #################
INFO:src.chat_bot:--   No relevant definitions found
INFO:src.chat_bot:--   Relevant Sections
INFO:src.chat_bot:0.1318:                C.(C):         heading: Acquisition of gold for trade purposes
INFO:src.chat_bot:0.1489:                C.(G):         heading: Applications for the importation of gold
INFO:src.chat_bot:Top result: C.(C) with a cosine distance of 0.13176155036104042
INFO:src.chat_bot:No unique mode
INFO:src.chat_bot:Only the top result added but more were found. Adding the next most likely answer
INFO:src.chat_bot:--   Filtered Sections
INFO:src.chat_bot:0.1318            :                C.(C):  1
INFO:src.chat_bot:0.1489            :                C.(G):  1


In [49]:
relevant_sections

,reference,cosine_distance,count,raw_text,token_count
0,C.(C),0.131650,1,C. Gold\n (C) Acquisition of gold for trade...,119
1,C.(G),0.148842,1,C. Gold\n (G) Applications for the importat...,43


In [5]:
excon_test.messages = [{"role": "user", "content": "Who can trade gold?"}]

flag, response = excon_test.resource_augmented_query(model_to_use="gpt-3.5-turbo", 
                                                     temperature = 0, 
                                                     max_tokens = 200, 
                                                     df_definitions = relevant_definitions, 
                                                     df_search_sections = relevant_sections)

INFO:src.chat_bot:#################   RAG       #################
INFO:src.chat_bot:
You are attempting to answer questions from an Authorised Dealer based only on the relevant documentation provided. You have only three options:
1) Answer the question. If you do this, your must preface to response with the word 'ANSWER:'. If possible also provide a reference to the relevant documentation for the user to cross-check. Use this if you are sure about your answer.
2) Request additional documentation. If, in the body of the relevant documentation, is a reference to another section of the document that is directly relevant, respond with the word 'SECTION:' followed by the section reference which looks like A.1(A)(i)(aa). 
3) State 'NONE:' (and nothing else) in all cases where you are not confident about either of the first two options
Potentially relevant document section(s):
C. Gold
    (C) Acquisition of gold for trade purposes
        (i) The acquisition of gold for legitimate trade purpo

In [6]:
print(flag)
print(response)

ANSWER:
 The acquisition of gold for legitimate trade purposes, such as by manufacturing jewellers or dentists, is subject to the approval of the South African Diamond and Precious Metals Regulator. After receiving such approval, a permit must be obtained from SARS, which will allow the permit holder to approach Rand Refinery Limited for an allocation of gold. The holders of gold, having received the necessary approvals, are exempt from certain provisions of Regulation 5(1). (Reference: C. Gold (C)(i))


In [6]:
import pandas as pd
text = "I.2 Local facilities to non-residents\n\
        (A) Short-term trade finance extended by a local Authorised Dealer to a non-resident directly related to the export of goods or services from South Africa\n\
            (i) In order to facilitate the export of goods from South Africa, the Financial Surveillance Department is prepared to permit local Authorised Dealers, on application, to extend short-term finance facilities denominated in Rand or a foreign currency to the non-resident importer or his/her banker abroad on the conditions mentioned below.\n\
                (a) Finance facilities denominated in Rand\n\
                    (aa) The facilities must relate to the post-shipment finance of the export of a specific shipment of goods and at the time of drawdown, the local Authorised Dealer extending these facilities must view the prescribed SARS Customs Declaration.\n\
                    (bb) The foreign currency proceeds eventually received in South Africa and/or payments received from any Non-resident Rand account, must be utilised to liquidate the aforementioned short-term export finance facility. The relevant Financial Surveillance Department requirements relating to the reporting in terms of the Reporting System and the observance of the 30-day rule must be complied with.\n\
                    (cc) The overall finance period, including any initial credit granted by the exporter, may not exceed six months from date of shipment of the underlying goods from South Africa unless the dispensation outlined in section B.18(B)(i)(b) of the Authorised Dealer Manual has been granted, when the overall finance period, including any initial credit granted by the exporter, may not exceed 12 months from date of shipment of the underlying goods from South Africa. An export finance facility may be extended in the event of the overseas importer requiring an extension of the original credit period, provided that the overall finance periods set out above are not exceeded.\n\
                    (dd) The facility must be repaid with foreign currency and/or by payment from a Non-resident Rand account. No facility may be drawn down where the proceeds of the underlying export transaction have already been received."
manual_data = ["I.2(A)(i)(a)", 1, text, 100]
df_manual_data = pd.DataFrame([manual_data], columns = ["reference", "count", "raw_text", "token_count"])

In [10]:
question = "Can you list the dispensations necessary for a rand facility to a non-resident exceed 6 months?"
excon_test.messages = [{"role": "user", "content": question}]
flag, response = excon_test.resource_augmented_query(model_to_use="gpt-4", #"gpt-3.5-turbo", 
                                                     temperature = 0, 
                                                     max_tokens = 200, 
                                                     df_definitions = pd.DataFrame(), 
                                                     df_search_sections = df_manual_data)

In [11]:
print(flag)
print(response)

SECTION:
 B.18(B)(i)(b)


In [14]:
section_to_add = response
section_to_add = section_to_add.strip()
section_to_add

'B.18(B)(i)(b)'

In [18]:
import regex

def find_fuzzy_reference(section_details, reference):
    # Remove special regex characters from the reference string and escape it
    sanitized_reference = regex.sub(r'[\\.*+?^${}()|[\]]', '', reference)
    pattern = r'(%s){e<=2}' % sanitized_reference
    match = regex.search(pattern, section_details)
    if match:
        return match.group()
    else:
        return None

In [20]:
text = "I.2 Local facilities to non-residents\n\
        (A) Short-term trade finance extended by a local Authorised Dealer to a non-resident directly related to the export of goods or services from South Africa\n\
            (i) In order to facilitate the export of goods from South Africa, the Financial Surveillance Department is prepared to permit local Authorised Dealers, on application, to extend short-term finance facilities denominated in Rand or a foreign currency to the non-resident importer or his/her banker abroad on the conditions mentioned below.\n\
                (a) Finance facilities denominated in Rand\n\
                    (aa) The facilities must relate to the post-shipment finance of the export of a specific shipment of goods and at the time of drawdown, the local Authorised Dealer extending these facilities must view the prescribed SARS Customs Declaration.\n\
                    (bb) The foreign currency proceeds eventually received in South Africa and/or payments received from any Non-resident Rand account, must be utilised to liquidate the aforementioned short-term export finance facility. The relevant Financial Surveillance Department requirements relating to the reporting in terms of the Reporting System and the observance of the 30-day rule must be complied with.\n\
                    (cc) The overall finance period, including any initial credit granted by the exporter, may not exceed six months from date of shipment of the underlying goods from South Africa unless the dispensation outlined in section B.18(B)(i)(b) of the Authorised Dealer Manual has been granted, when the overall finance period, including any initial credit granted by the exporter, may not exceed 12 months from date of shipment of the underlying goods from South Africa. An export finance facility may be extended in the event of the overseas importer requiring an extension of the original credit period, provided that the overall finance periods set out above are not exceeded.\n\
                    (dd) The facility must be repaid with foreign currency and/or by payment from a Non-resident Rand account. No facility may be drawn down where the proceeds of the underlying export transaction have already been received."
        
section = "B.18 Gold (B)(i)(b)"
match = find_fuzzy_reference(text, section)
print(match)

None


In [17]:
type(match)

str

In [31]:
import re

def extract_valid_reference(input_string):
    excon_index_patterns = [
        r'[A-Z]',
        r'\.\d{1,2}',
        r'\([A-Z]\)',
        r'\((i|ii|iii|iv|v|vi|vii|viii|ix|x|xi|xii|xiii|xiv|xv|xvi|xvii|xviii|xix|xx|xxi|xxii|xxiii)\)',
        r'\([a-z]\)',
        r'\([a-z]{2}\)',
        r'\((?:[1-9]|[1-9][0-9])\)'
    ]
    
    partial_ref = ""
    remaining_str = input_string
    
    for pattern in excon_index_patterns:
        match = re.search(pattern, remaining_str)
        if match:
            partial_ref += match.group()
            remaining_str = remaining_str[match.end():]
        else:
            if remaining_str:
                return None
            break
    
    return partial_ref if partial_ref else None

In [32]:
# Test the function
print(extract_valid_reference('B.18 Gold (B)(i)(b)'))  # Output: 'B.18(B)(i)(b)'
print(extract_valid_reference('B.18 Gold (B)(a)(b)'))  # Output: None
print(extract_valid_reference('A.1'))  # Output: 'A.1'

B.18(B)(i)(b)
None
A.1
